In [ ]:
import numpy as np
import matplotlib.pylab as plt
import uproot
import awkward as ak

import vector

import sys

vector.register_awkward()

import coffea

from coffea.nanoevents import NanoEventsFactory

import itertools
from itertools import combinations

In [ ]:
print(f"python: {sys.version}\n")

print(f"numpy:   {np.__version__}")
print(f"uproot:  {uproot.__version__}")
print(f"awkward: {ak.__version__}")
print(f"vector:  {vector.__version__}")
print()

print(f"coffea:  {coffea.__version__}")


In [ ]:
data_dir = '/home/bellis/top_data/NANOAOD/'

# Laptop
#infilename = 'small_skims_10k/TT_TToBCE_TuneCP5_BNV_2018_SMALL_10k.root'
#infilename = 'TTToHadronic_UL_2018_SMALL_100k.root'

# Beth Harmon
#infilename = 'small_skims_1k/TTbarPowheg_Hadronic_2017_SMALL_1k.root'
#infilename = 'TTToSemiLeptonic_UL_2018.root' # Also desktop home computer
#infilename = 'Reza_signal/nAOD_step_BNV_TT_TSUE/NAOD-00000_190.root' # Also desktop home computer

# Desktop home computer
infilename = 'small_skims_1k/TTbarPowheg_Hadronic_2017_SMALL_1k.root'



infile = uproot.open(data_dir + infilename)


In [ ]:
################################################################################
# Generate the indices for the diferent combinations
################################################################################
def generate_event_topology_indices(njets,nleps,verbose=False):

    index_combinations = []

    if njets<5 or nleps<1:
        return [[None,None,None,None,None,None]]

    jetindices = np.arange(njets,dtype=int)
    lepindices = np.arange(nleps,dtype=int)

    x = combinations(jetindices,3)

    for had in x:
        remaining = np.delete(jetindices, np.argwhere( (jetindices==had[0]) | (jetindices==had[1]) | (jetindices==had[2]) ))
        bnv = combinations(remaining,2)
        for b in bnv:
            for lep in lepindices:
                index_combinations.append([had[0],had[1],had[2],b[0],b[1],lep])

                if verbose:
                    print(had,b,lep)

    return np.array(index_combinations)


In [ ]:
#-------------------------------#

njet = infile["Events"]["nJet"].array()#[0:1000]
nmuon = infile["Events"]["nMuon"].array()#[0:1000]

min_njets = 5
max_njets = 8

mask_njets = (njet>=min_njets) & (njet<=max_njets)
#-------------------------------#

muon_branch_arrays = infile["Events"].arrays(filter_name="Muon_*")#[0:1000]
jet_branch_arrays = infile["Events"].arrays(filter_name="Jet_*")#[0:1000]

muons = ak.zip({
    "pt": muon_branch_arrays[mask_njets]["Muon_pt"],
    "phi": muon_branch_arrays[mask_njets]["Muon_phi"],
    "eta": muon_branch_arrays[mask_njets]["Muon_eta"],
    "mass": muon_branch_arrays[mask_njets]["Muon_mass"],
    "charge": muon_branch_arrays[mask_njets]["Muon_charge"],
}, with_name="Momentum4D")

jets = ak.zip({
    "pt": jet_branch_arrays[mask_njets]["Jet_pt"],
    "phi": jet_branch_arrays[mask_njets]["Jet_phi"],
    "eta": jet_branch_arrays[mask_njets]["Jet_eta"],
    "mass": jet_branch_arrays[mask_njets]["Jet_mass"],
    "btag": jet_branch_arrays[mask_njets]["Jet_btagDeepB"],
}, with_name="Momentum4D")

jet_combos = ak.combinations(jets, 5)
#muon_combos = ak.combinations(muons, 1)
muon_combos = ak.zip((muons,))

jet1, jet2, jet3, jet4, jet5 = ak.unzip(jet_combos)
muon1 = ak.unzip(muon_combos)

### Need this for later

njet_masked = infile["Events"]["nJet"].array()[mask_njets]#[0:1000]
nmuon_masked = infile["Events"]["nMuon"].array()[mask_njets]#[0:1000]

In [ ]:
print(len(njet), len(njet_masked))

In [ ]:
# Generate the combinations

combos = []
event_indices = []

icount = 0
for i,(nj,nm) in enumerate(zip(njet_masked,nmuon_masked)):
    
    combo = generate_event_topology_indices(njets=nj,nleps=nm)

    if combo[0][0] is None:
        continue
    
    for c in combo:
        combos.append(np.array(c))
    event_indices += (i*np.ones(len(combo),dtype=int)).tolist()
    icount += 1

combos = np.array(combos)

print(combos.shape)

In [ ]:
jet1 = jets[(event_indices,combos.transpose()[0])]
jet2 = jets[(event_indices,combos.transpose()[1])]
jet3 = jets[(event_indices,combos.transpose()[2])]
jet4 = jets[(event_indices,combos.transpose()[3])]
jet5 = jets[(event_indices,combos.transpose()[4])]

muon = muons[(event_indices,combos.transpose()[5])]

In [ ]:
def top_variables(jets, decay_type='had'):    
    had_dR12 = jets[0].deltaR(jets[1])
    had_dR13 = jets[0].deltaR(jets[2])
    had_dR23 = jets[1].deltaR(jets[2])
    had_dR1_23 = jets[0].deltaR(jets[1] + jets[2])
    
    had_p12 = jets[0] + jets[1]
    had_p13 = jets[0] + jets[2]
    had_p23 = jets[1] + jets[2]
    
    # btag
    var1 = jets[0].btag
    var2 = jets[1].btag
    var3 = None
    if decay_type == 'had':
        var3 = jets[2].btag
    elif decay_type == 'bnv':
        var3 = jets[2].charge
        
    # Boost
    top_p4 = jets[0] + jets[1] + jets[2]
    boost_p4 = top_p4
    
    boost_p4 = ak.with_field(boost_p4, 173.0, "tau")

    jet1_boosted = jets[0].boostCM_of(boost_p4)
    jet2_boosted = jets[1].boostCM_of(boost_p4)
    jet3_boosted = jets[2].boostCM_of(boost_p4)
    
    had_dThetaCM12 = jet1_boosted.deltaangle(jet2_boosted)
    had_dThetaCM13 = jet1_boosted.deltaangle(jet3_boosted)
    had_dThetaCM23 = jet2_boosted.deltaangle(jet3_boosted)
    had_dThetaCM1_23 = jet1_boosted.deltaangle(jet2_boosted + jet3_boosted)
    
    # Sort things by the magnitude of momentum in the top-CM frame    
    sort_by = np.array([jet1_boosted.mag, jet2_boosted.mag, jet3_boosted.mag]).transpose()
    idx = np.argsort(sort_by)
    array_to_sort = sort_by
    jet3_boosted_mag,jet2_boosted_mag,jet1_boosted_mag = np.take_along_axis(array_to_sort, idx, axis=1).transpose()
    
    array_to_sort = np.array([jet1.pt, jet2.pt,jet3.pt]).transpose()
    jet3_pt,jet2_pt,jet1_pt = np.take_along_axis(array_to_sort, idx, axis=1).transpose()

    array_to_sort = np.array([jet1_boosted.pt, jet2_boosted.pt,jet3_boosted.pt]).transpose()
    jet3_boosted_pt,jet2_boosted_pt,jet1_boosted_pt = np.take_along_axis(array_to_sort, idx, axis=1).transpose()

    array_to_sort = np.array([var1, var2, var3]).transpose()
    var3, var2, var1 = np.take_along_axis(array_to_sort, idx, axis=1).transpose()

    
    #xsorted,ysorted,zsorted = xsort.transpose()
    
    return top_p4.mass, had_p12.mass, had_p13.mass, had_p23.mass, \
           had_dR12, had_dR13, had_dR23, had_dR1_23, \
           had_dThetaCM12, had_dThetaCM13, had_dThetaCM23, had_dThetaCM1_23, \
           jet1_boosted_pt, jet2_boosted_pt, jet3_boosted_pt, \
           jet1_boosted_mag, jet2_boosted_mag, jet3_boosted_mag, \
           var1, var2, var3


def event_hypothesis(jets, lepton):
    had_p4 = jets[0] + jets[1] + jets[2]
    bnv_p4 = jets[3] + jets[4] + lepton
    
    angle = had_p4.deltaangle(bnv_p4)

    had_variables = top_variables([jets[0], jets[1], jets[2]], decay_type='had')
    bnv_variables = top_variables([jets[3], jets[4], lepton], decay_type='bnv')
    
    return had_variables, bnv_variables, angle



In [ ]:
had_variables, bnv_variables, angle = event_hypothesis([jet1, jet2, jet3, jet4, jet5], muon)


In [ ]:
#values[2]

In [ ]:
for values in [had_variables, bnv_variables]:
    plt.figure(figsize=(16,12))

    for i in range(len(values)):
        #x = ak.flatten(values[i]).to_numpy()   
        #print(type(values[i]))
        x = values[i]
        if type(x) == ak.highlevel.Array:
            x = values[i].to_numpy()
        #print(type(x))

        #print(len(x),x)
        #print(len(x[x==x]))
        x[x==-np.inf] = -999
        x[x==np.inf] = -999
        plt.subplot(5,5,i+1)
        if i>=15 and i<=17:
            plt.hist(x[x==x],bins=100,range=(0,250))
        else:
            plt.hist(x[x==x],bins=100)

plt.figure()
plt.hist(np.cos(angle),bins=100);

In [ ]:
plt.hist(np.unique(had_variables[-3]),bins=100,range=(-3,3));#, range=(0,1200));

In [ ]:
plt.hist(had_variables[-6],bins=100,range=(0,250));
plt.hist(had_variables[-5],bins=100,range=(0,250),alpha=0.6);
plt.hist(had_variables[-4],bins=100,range=(0,250),alpha=0.4);

In [ ]:
plt.hist(had_variables[-9],bins=100,range=(0,150));
plt.hist(had_variables[-8],bins=100,range=(0,150),alpha=0.6);
plt.hist(had_variables[-7],bins=100,range=(0,150),alpha=0.4);

In [ ]:
x = values[0]

In [ ]:
x.to_numpy()

In [ ]:
x = ak.Array([12, 14, 19, 16]).to_numpy()
y = ak.Array([10, 15, 18, 13]).to_numpy()

In [ ]:
z = np.array([x,y])

In [ ]:
zt = z.transpose()

In [ ]:
idx = np.argsort(zt).transpose()

In [ ]:
idx

In [ ]:
x[idx[0]==0]

In [ ]:
pt1 = had_variables[-6].to_numpy()
pt2 = had_variables[-5].to_numpy()
pt3 = had_variables[-4].to_numpy()

boosted_pts = np.array([pt1, pt2, pt3])

In [ ]:
idx = np.argsort(boosted_pts.transpose()).transpose()

idx[0]==0

In [ ]:
idx[0][idx[0]==1]

In [ ]:
sort_pt1 = []
sort_pt2 = []
sort_pt3 = []

sort_pt3 += pt1[idx[0]==0].tolist()
sort_pt3 += pt2[idx[1]==0].tolist()
sort_pt3 += pt3[idx[2]==0].tolist()

sort_pt2 += pt1[idx[0]==1].tolist()
sort_pt2 += pt2[idx[1]==1].tolist()
sort_pt2 += pt3[idx[2]==1].tolist()

sort_pt1 += pt1[idx[0]==2].tolist()
sort_pt1 += pt2[idx[1]==2].tolist()
sort_pt1+= pt3[idx[2]==2].tolist()

In [ ]:
pt1[idx[0]==2]

In [ ]:
plt.hist(sort_pt1,bins=100,range=(0,250));
plt.hist(sort_pt2,bins=100,range=(0,250),alpha=0.4);
plt.hist(sort_pt3,bins=100,range=(0,250),alpha=0.4);

In [ ]:
m12 = had_variables[1].to_numpy()
m13 = had_variables[2].to_numpy()
m23 = had_variables[3].to_numpy()

In [ ]:
# 1 2 3
# 2 3 1
# 3 1 2

# 1 3 2
# 2 1 3
# 3 2 1


In [ ]:
m12_sorted = []
m13_sorted = []
m23_sorted = []

print(len(m12))

tot = 0

mask = (idx[0]==2) & (idx[1]==1)
tot += len(m12[mask])
m12_sorted += m12[mask].tolist()

mask = (idx[1]==2) & (idx[0]==1)
tot += len(m12[mask])
m12_sorted += m12[mask].tolist()


mask = (idx[0]==2) & (idx[2]==1)
tot += len(m13[mask])
m12_sorted += m13[mask].tolist()

mask = (idx[2]==2) & (idx[0]==1)
tot += len(m13[mask])
m12_sorted += m13[mask].tolist()

mask = (idx[1]==2) & (idx[2]==1)
tot += len(m23[mask])
m12_sorted += m23[mask].tolist()

mask = (idx[2]==2) & (idx[1]==1)
tot += len(m23[mask])
m12_sorted += m23[mask].tolist()


print(tot)


print(len(m13))

tot = 0

mask = idx[2]==0
tot += len(m12[mask])
m12_sorted += m12[mask].tolist()

mask = idx[1]==0
tot += len(m13[mask])
m12_sorted += m13[mask].tolist()

mask = idx[0]==0
tot += len(m23[mask])
m12_sorted += m23[mask].tolist()



print(tot)

In [ ]:
plt.hist(m12_sorted,bins=100,range=(0,250));

In [ ]:
jet1.pt

In [ ]:
jet1.rho

In [ ]:
print(jet1.pt[0], jet1.eta[0], jet1.phi[0], jet1.mag[0], jet1.energy[0])

# mag is magnitude of 3-momentum

In [ ]:
print(jet1.pt[1], jet1.eta[1], jet1.phi[1], jet1.mag[1], jet1.energy[1])

In [ ]:
#for N in range(1000,1100):
#    print(jet1.pt[N], jet1.eta[N], jet1.phi[N], jet1.mag[N], jet1.energy[N], jet1.tau[N])